In [1]:
# 1. Install the libraries in the Colab environment
# (This is like 'pip install' but for Colab)
!pip install transformers datasets evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 13.1 MB/s eta 0:00:00


In [3]:
# 2. Import the main 'load_dataset' function
from datasets import load_dataset

In [9]:
# 3. Load dataset
# This dataset is in the modern (script-free) format and is already pre-processed in a way that's perfect for Text-to-SQL.
dataset = load_dataset("b-mc2/sql-create-context")

# 4. Inspect the dataset
print("--- Dataset Structure ---")
print(dataset)
print("\n--- Example from Training Set ---")
print(dataset['train'][0])

README.md: 0.00B [00:00, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

--- Dataset Structure ---
DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 78577
    })
})

--- Example from Training Set ---
{'answer': 'SELECT COUNT(*) FROM head WHERE age > 56', 'question': 'How many heads of the departments are older than 56 ?', 'context': 'CREATE TABLE head (age INTEGER)'}


In [10]:
# 1. Split the original 'train' set into 80% train and 20% temporary test
train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=42) # seed for reproducible results

# 2. Rename the new 'train' set
dataset['train'] = train_test_split['train']

# 3. Split the 20% temporary test set into 10% validation and 10% test (50/50 split)
validation_test_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

# 4. Assign the final validation and test sets
dataset['validation'] = validation_test_split['train']
dataset['test'] = validation_test_split['test']

# 5. Let's check our new dataset structure
print("--- NEW Dataset Structure ---")
print(dataset)

--- NEW Dataset Structure ---
DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 62861
    })
    validation: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
})


In [11]:
# 1. Import the tokenizer for our model (t5-small)
from transformers import T5Tokenizer

model_checkpoint = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

# 2. Define the prefix we'll use for every input
prefix = "translate to SQL: "

# 3. Create the preprocessing function
def preprocess_function(examples):
    # 3a. Build the full input string
    inputs = [prefix + "question: " + q + " | context: " + c for q, c in zip(examples["question"], examples["context"])]

    # 3b. Tokenize the input strings
    # max_length=512 is a good starting point
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # 3c. Tokenize the target (SQL query)
    # We use 'as_target_tokenizer()' to set up the labels correctly
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["answer"], max_length=128, truncation=True, padding="max_length")

    # 3d. Set the tokenized labels
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 4. Apply this function to our entire dataset!
# We use batched=True to speed this up significantly
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# 5. Check our work
print("\n--- Example of a Tokenized Input ---")
print(tokenized_dataset['train'][0])

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]


--- Example of a Tokenized Input ---
{'answer': 'SELECT perth FROM table_name_56 WHERE gold_coast = "yes" AND sydney = "yes" AND melbourne = "yes" AND adelaide = "yes"', 'question': 'Which Perth has Gold Coast yes, Sydney yes, Melbourne yes, and Adelaide yes?', 'context': 'CREATE TABLE table_name_56 (perth VARCHAR, adelaide VARCHAR, melbourne VARCHAR, gold_coast VARCHAR, sydney VARCHAR)', 'input_ids': [13959, 12, 12558, 10, 822, 10, 4073, 22343, 65, 2540, 5458, 4273, 6, 7476, 4273, 6, 9396, 4273, 6, 11, 24272, 4273, 58, 1820, 2625, 10, 205, 4386, 6048, 332, 17098, 953, 834, 4350, 834, 4834, 41, 883, 189, 584, 4280, 28027, 6, 3, 15311, 5385, 584, 4280, 28027, 6, 3, 2341, 26255, 584, 4280, 28027, 6, 2045, 834, 25500, 584, 4280, 28027, 6, 3, 7, 63, 26, 3186, 584, 4280, 28027, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [17]:
# --- PLAN D (Manually adding 'generation_config') ---

from transformers import (
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Seq2SeqTrainer,
    GenerationConfig  # <-- We need to import this
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Calculate steps per epoch
train_dataset_size = len(tokenized_dataset['train'])
our_batch_size = 8
steps_per_epoch = train_dataset_size // our_batch_size

# 1. Define TrainingArguments as before
training_args = TrainingArguments(
    output_dir="./results",
    eval_steps=steps_per_epoch,
    save_steps=steps_per_epoch,
    logging_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=our_batch_size,
    per_device_eval_batch_size=our_batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    fp16=True,
    push_to_hub=False,
)

# --- THIS IS THE FIX ---
# 2. Manually load the GenerationConfig from the model
gen_config = GenerationConfig.from_pretrained(model_checkpoint)

# 3. Manually add the 'generation_config' attribute to our 'training_args'
# This fixes the AttributeError because 'self.args.generation_config' will now exist.
training_args.generation_config = gen_config
# -----------------------

# 4. Create the Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 5. Initialize the Trainer
# This should finally work without error
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("--- Starting Model Training ---")
trainer.train()
print("--- Model Training Complete ---")

print("--- Saving Final Model ---")
trainer.save_model("./my_final_text_to_sql_model")
print("--- Model Saved ---")

/tmp/ipython-input-325581419.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


--- Starting Model Training ---


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rishabhagrawal0623 (rishabhagrawal0623-kj-somaiya-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,1.017500
1000,0.160800
1500,0.131700
2000,0.118800
2500,0.104500
3000,0.096100
3500,0.087700
4000,0.084600
4500,0.080000
5000,0.076500


--- Model Training Complete ---
--- Saving Final Model ---
--- Model Saved ---


In [18]:
# This will create a zip file named 'my_model.zip' containing your entire trained model
print("--- Zipping model folder... ---")
!zip -r /content/my_model.zip /content/my_final_text_to_sql_model
print("--- Zipping complete! ---")

--- Zipping model folder... ---
  adding: content/my_final_text_to_sql_model/ (stored 0%)
  adding: content/my_final_text_to_sql_model/tokenizer_config.json (deflated 94%)
  adding: content/my_final_text_to_sql_model/added_tokens.json (deflated 83%)
  adding: content/my_final_text_to_sql_model/spiece.model (deflated 48%)
  adding: content/my_final_text_to_sql_model/generation_config.json (deflated 27%)
  adding: content/my_final_text_to_sql_model/special_tokens_map.json (deflated 85%)
  adding: content/my_final_text_to_sql_model/model.safetensors (deflated 9%)
  adding: content/my_final_text_to_sql_model/config.json (deflated 63%)
  adding: content/my_final_text_to_sql_model/training_args.bin (deflated 54%)
--- Zipping complete! ---


In [19]:
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer

# 1. Load the tokenizer and our fine-tuned model
model_path = "./my_final_text_to_sql_model"
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# 2. Select a few examples from our test set
test_examples = dataset['test'].select(range(5)) # Let's test the first 5

# 3. Define the prefix (must be the same as in training)
prefix = "translate to SQL: "

# 4. Loop through the examples and test our model
for example in test_examples:
    question = example["question"]
    context = example["context"]
    ground_truth_sql = example["answer"]

    # 4a. Format the input
    input_text = f"{prefix}question: {question} | context: {context}"

    # 4b. Tokenize the input
    input_ids = tokenizer(
        input_text,
        return_tensors="pt", # Return PyTorch tensors
        max_length=512,
        truncation=True
    ).input_ids

    # 4c. Generate the SQL query
    # .generate() is the core function for text generation
    outputs = model.generate(
        input_ids,
        max_length=128,      # Max length of the generated SQL
        num_beams=4,         # Use beam search for better results
        early_stopping=True
    )

    # 4d. Decode the generated token IDs back into a string
    predicted_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 5. Print the results
    print("-" * 50)
    print(f"QUESTION:\n{question}")
    print(f"\nGROUND TRUTH SQL:\n{ground_truth_sql}")
    print(f"\nMODEL'S PREDICTED SQL:\n{predicted_sql}")
    print("-" * 50)

--------------------------------------------------
QUESTION:
Which qual has both 200 total laps and took place in 1957?

GROUND TRUTH SQL:
SELECT qual FROM table_name_80 WHERE laps = 200 AND year = "1957"

MODEL'S PREDICTED SQL:
SELECT qual FROM table_name_80 WHERE laps = "200" AND year = 1957
--------------------------------------------------
--------------------------------------------------
QUESTION:
What is the location of the gold medal match?

GROUND TRUTH SQL:
SELECT location FROM table_name_30 WHERE competition = "gold medal match"

MODEL'S PREDICTED SQL:
SELECT location FROM table_name_30 WHERE competition = "gold medal match"
--------------------------------------------------
--------------------------------------------------
QUESTION:
What was the total number of wins with player Mike Hill with a rank bigger than 2?

GROUND TRUTH SQL:
SELECT COUNT(wins) FROM table_name_73 WHERE rank > 2 AND player = "mike hill"

MODEL'S PREDICTED SQL:
SELECT COUNT(wins) FROM table_name_73 WH

In [2]:
!pip install sentencepiece tqdm

In [10]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer
import torch
import os # We'll use this to check the path

# --- 1. UNZIP YOUR MODEL ---
# We add -o to overwrite if it's already there
print("--- Unzipping model... ---")
!unzip -q -o my_model.zip
print("--- Unzip complete. ---")

# --- 2. LOAD DATASET ---
print("--- Loading dataset from Hugging Face... ---")
dataset = load_dataset("b-mc2/sql-create-context")
train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=42)
validation_test_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)
test_dataset = validation_test_split['test']
print(f"--- Loaded {len(test_dataset)} test examples ---")


# --- 3. LOAD YOUR UPLOADED MODEL ---
print("--- Loading local model... ---")

# --- THIS IS THE NEW FIX ---
# We are using the FULL absolute path in Colab
model_path = "/content/content/my_final_text_to_sql_model"
# --------------------------

# Check if the folder actually exists
if not os.path.exists(model_path):
    print("="*50)
    print(f"ERROR: Model folder not found at {model_path}")
    print("Please make sure you uploaded 'my_model.zip' successfully.")
    print("="*50)
else:
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    prefix = "translate to SQL: "

    # --- 4. MOVE MODEL TO GPU ---
    model.to("cuda") # This moves the model to the T4 GPU
    print("--- Model loaded and moved to GPU! ---")

--- Unzipping model... ---
--- Unzip complete. ---
--- Loading dataset from Hugging Face... ---
--- Loaded 7858 test examples ---
--- Loading local model... ---
--- Model loaded and moved to GPU! ---


In [11]:
import evaluate
from tqdm import tqdm # A progress bar library

# --- 1. LOAD METRIC AND DATA ---
print("--- Preparing for evaluation... ---")
bleu = evaluate.load("bleu")
references = [[ex["answer"]] for ex in test_dataset]
predictions = []

# --- 2. GENERATE PREDICTIONS ---
print("--- Generating predictions for the test set (on GPU)... ---")

# Use tqdm for a nice progress bar
for example in tqdm(test_dataset, desc="Generating Predictions"):
    question = example["question"]
    context = example["context"]

    # Format and tokenize
    input_text = f"{prefix}question: {question} | context: {context}"
    input_ids = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=512,
        truncation=True
    ).input_ids

    # Move the input data to the GPU
    input_ids = input_ids.to(model.device)

    # Generate
    outputs = model.generate(
        input_ids,
        max_length=128,
        num_beams=4,
        early_stopping=True
    )

    # Decode and add to our list
    predicted_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions.append(predicted_sql)

# --- 3. COMPUTE METRICS ---
print("\n--- Computing BLEU score ---")
results = bleu.compute(predictions=predictions, references=references)

# Calculate Exact Match
exact_matches = 0
for pred, ref in zip(predictions, references):
    if pred == ref[0]:
        exact_matches += 1

# --- 4. PRINT FINAL METRICS ---
print("\n" + "="*30)
print("     EVALUATION METRICS     ")
print("="*30)
print(f"Total Test Examples:  {len(predictions)}")
print(f"BLEU Score:           {results['bleu'] * 100:.2f}")
print(f"Exact Match Accuracy: {(exact_matches / len(predictions)) * 100:.2f}%")
print("="*30)

--- Preparing for evaluation... ---


--- Generating predictions for the test set (on GPU)... ---


Generating Predictions: 100%|██████████| 7858/7858 [46:59<00:00,  2.79it/s]



--- Computing BLEU score ---

     EVALUATION METRICS     
Total Test Examples:  7858
BLEU Score:           88.37
Exact Match Accuracy: 50.00%
